<div style="text-align: center; margin: 50px">

<h1 style="color: white; background-color: grey; text-align: center;">Qubit by Qubit - Semester 2</h1>
<h3>Week 19 lab: Quantum Key Distribution</h3>

</div>

# Overview

1. [Introduction](#intro) <br>
    1.1 [Activity layout](#layout) <br>
    1.2 [Experiment specifications](#specs) <br>
2. [Steps in the QKD protocol](#protocol) <br>
    2.1 [Step 1: Select Encoding](#step1) <br>
    2.2 [Step 2: Select Measurement](#step2) <br>
    2.3 [Step 3: Encode](#step3)  <br>
    2.4 [Step 4: Send](#step4)  <br>
3. [Key Takeaways](#takeaways)  <br>
4. [Further readings and resources](#frr)  <br>
5. [Optional content](#optional) <br>
    5.1 [Step 5: Measure](#step5) <br>
    5.2 [Step 6: Announce bases](#step6) <br>
    5.3 [Step 7: Find symmetric key](#step7) <br>
6. [Exta content - full QKD protocol implementation](#extra) <br>
    6.1 [Challenge problem: Implementing a function for Eve](#challenge) <br>

<a id="intro"></a>
# 1. Introduction

**What is key distribution?** Secure communication relies on the ability of the sender (Alice) to encrypt the message in a way that the receiver (Bob) can decrypt it but not an eavesdropper. This security is often accomplished with the use of a **key**, which is a piece of information known only to the sender and receiver and enables them to decrypt and encrypt the message. If a key can be securely distributed between the sender and receiver, the encrypted message can be securely sent over a public channel, since without the key, the probability of successfully decrypting the message is tiny.

Practically, **a key is just a bitstring** - a sequence of 1s and 0s, that is uniquely known only to Alice and Bob, the two communicating parties.

Therefore, the problem of secure communication boils down to secure key distribution. QKD is unique because security against eavesdropping is guaranteed by the laws of quantum mechanics, as opposed to the computational complexity of certain functions, which is what is used in classical key distribution. 

In this lab, we will develop qiskit code to implement Quantum Key Distribution (QKD). We will look at the same BB84 protocol that was discussed in lecture this week. 


We will also continue making the conceptual leap we started last week - instead of directly defining circuits, we will create functions that define the circuits for us. **We're learning how to move up the quantum stack!**

<a id="layout"></a>
## 1.1 Quantum Key Distribution Activity layout

In this lab, and in your homework, we are going to implement the BB84 protocol. Remember from lecture the following steps of the protocol:

1. `SELECT ENCODING`: Alice randomly selects a basis ( × or + ) to encode each bit.
2. `SELECT MEASUREMENT`: Bob randomly selects a basis ( × or + ) to measure each bit.
3. `ENCODE`: Alice creates the quantum states, encoded in the selected bases.
4. `SEND`: Alice sends Bob the encoded states, via the quantum channel. 
5. `MEASURE`: Bob measures all the quantum states in his pre-selected measurement bases.
6. `ANNOUNCE BASIS`: Alice announces which basis she used to encode each bit, via the classical channel.
7. `FIND SYMMETRIC KEY`: Alice and Bob discard bits in their key that used a different encoding and decoding basis.

These 7 steps allow a key to be distributed between Alice and Bob securely, now the two can send secure and encrypted messages through an insecure channel. In the required content for this lab, we will implement steps 1-4, and in the optional content we will implement steps 5-7.

Note that we have left out **Step8: Analysis** here. In this lab, we will not worry about an eavesdropper, but focus on the code for the basic protocol. Therefore, Alice and Bob don't need to run an analysis step. If you're interested, you can try implementing code for Eve in the optional challenge section at the end of the notebook!

In [1]:
from random import getrandbits
import qiskit as q

<a id="specs"></a>
## 1.2 Experiment specifications

In this excercise we are going to use a key length of 500, you can see this defined below as `KEY_LENGTH = 500`

In a real implemenetation of this, the physical photons will be sent down an optical fibre to Bob in step 4. Unfortunately, we can't actually do this in this lab, so instead we can represent the photons being transported by putting them in a `QUANTUM_CHANNEL` object - it's just a python list here.

Similarily we will need a representation of the classical channel which in real life would be physical wires sending electrical signals, over which Alice will announce her bases in step 6. We can represent this using a `CLASSICAL_CHANNEL` object, also just a list here!

We are also supposing that the `CLASSICAL_CHANNEL` is unsecure and therefore open to eavesdropping.

In [41]:
KEY_LENGTH = 500
QUANTUM_CHANNEL = []
CLASSICAL_CHANNEL = []

<a id="protocol"></a>
# 2. Steps in the QKD protocol

We will now implement the steps in the QKD protocol one-by-one.

<a id="step1"></a>
## 2.1. Step 1: Select Encoding

Alice needs to randomly select a bit key and a basis in which to encode each bit of the bit key.

This function takes in the number of bases that Alice needs to randomly pick and returns a list of each chosen encoding represented by either a 0 or a 1

In [42]:
def select_encoding(length):
    
    #This stores the states Alice will encode
    alice_bitstring = ""
    # This stores the bases that Alice will prepare the states in
    alice_bases = ""
    
    # For the length 
    for i in range(length):
        # We use the function getrandbits to get either a 0 or 1 randomly,
        # The "1" in the function argument is the number of bits to be generated
        alice_bitstring += (str(getrandbits(1)))
        # 0 means encode in the (0,1) basis and 1 means encode in the (+,-) basis
        alice_bases += (str(getrandbits(1)))
    
    # return the string of bits and the list of bases they should be encoded in
    return alice_bitstring, alice_bases

#### Create Alice's bit_key and alice_bases

Let's create our `alice_bitstring` and `alice_bases`

We can see what they look like by printing the first 10 elements, this 
should look like a string and a list of random '1's or '0's 

In [43]:
alice_bitstring, alice_bases = select_encoding(KEY_LENGTH)

# Preview the first 10 elements of each:
print("alice_bitstring: ", alice_bitstring[:10])
print("alice_bases: ", alice_bases[:10])

alice_bitstring:  1010001111
alice_bases:  1010000000


<a id="step2"></a>
## 2.2. Step 2: Select Measurement

Bob needs to randomly select a basis in which to measure each bit.

This function takes in the number of bases that Bob needs to randomly pick and returns a list of each chosen measurement basis represented by either a 0 or a 1

In [44]:
def select_measurement(length):
    # Similar to before we store the bases that Bob will measure in a list
    bob_bases = ""
    
    for i in range(length):
        # Again we use getrandbits to generate a 0 or 1 randomly
        bob_bases += (str(getrandbits(1)))
        
    # return the list of random bases to measure in
    return bob_bases

#### Create Bob's selected_measurements

Let's create our `bob_bases` list

We can see what it looks like by printing the first 10 elements, it should look like a list of random '1's or '0's 

In [45]:
bob_bases = select_measurement(KEY_LENGTH)

# Preview the first 10 elements of each:
print("selected_measurements: ", bob_bases[:10])

selected_measurements:  1000101010


<a id="step3"></a>
## 2.3. Step 3: Encode

Alice now uses her random list of numbers to generate a bunch of quantum states:
In this excercise we are going to represent the creation of a qubit as an individual `QuantumCircuit` object.

The table below summarizes the qubit states Alice sends, based on the bit of Alice's `alice_bitstring` the corresponding bit of `selected_bases`:

| Bit in Alice's `alice_bitstring` | Corresponding bit in `alice_bases` | Encoding basis | Qubit state sent |
|:----------------:|:--------------------------:|:--------------------------:|:---------------:|
| 0 | 0 | $$|0\rangle,|1\rangle$$ |$$|0\rangle$$ |
| 0 | 1 | $$|+\rangle,|-\rangle$$ |$$|+\rangle$$ |
| 1 | 0 | $$|0\rangle,|1\rangle$$ |$$|1\rangle$$ |
| 1 | 1 | $$|+\rangle,|-\rangle$$ |$$|-\rangle$$ |

In [46]:
def encode(alice_bitstring, alice_bases):
    encoded_qubits = []
    for i in range(len(alice_bitstring)):
        # create a brand new quantum circuit called qc. Remember that the qubit will be in state |0> by default
        qc = q.QuantumCircuit(1,1)

        if alice_bases[i] == "0":
            # 0 Means we are encoding in the z basis
            if alice_bitstring[i] == "0":
                # We want to encode a |0> state, as states are intialized
                # in |0> by default we don't need to add anything here
                pass
            
            elif alice_bitstring[i] == "1":
                # We want to encode a |1> state
                # We apply an X gate to generate |1>
                qc.x(0)
                
        elif alice_bases[i] == "1":
            # 1 Means we are encoding in the x basis
            if alice_bitstring[i] == "0":
                # We apply an H gate to generate |+>
                qc.h(0)
            elif alice_bitstring[i] == "1":
                # We apply an X and an H gate to generate |->
                qc.x(0)
                qc.h(0)
            
        # add this quantum circuit to the list of encoded_qubits
        encoded_qubits.append(qc)
        
    return encoded_qubits

### Create Alice's encoded_qubits

In [47]:
# Alice can now create the encoded qubits using the bit_key and selected_bases from 1
encoded_qubits = encode(alice_bitstring, alice_bases)

<a id="step4"></a>
## 2.4. Step 4: Send

Alice now sends the encoded qubits to Bob via the `QUANTUM_CHANNEL`. In a real application, Alice might send her qubits over an optical fibre, which would be the quantum channel. Here, we'll just copy the encoded qubits onto our variable `QUANTUM_CHANNEL`.

In [48]:
# Alice Sends Bob her encoded qubits over the quantum_channel
QUANTUM_CHANNEL = encoded_qubits

<a id="takeaways"></a>
# 3. Key takeaways

1. Key distribution is an essential step in secure communication - it ensures that Alice and Bob can encrypt and decrypt messages but an eavesdropper cannot.

2. QKD works in 3 steps: first, Alice generates a sequence of qubits and sends them to Bob. Next, Bob measures the received qubits. Finally, Alice and Bob compare their bases, and keep those bits where their bases match.

3. We can use python functions to code each of these steps to simulate QKD

<a id="frr"></a>
# 4. Further readings and resources

1. [Qiskit textbook page on QKD, including QKD with eavesdropping](https://qiskit.org/textbook/ch-algorithms/quantum-key-distribution.html)
2. [Prospects for large scale QKD](https://arxiv.org/ftp/arxiv/papers/1809/1809.02291.pdf)
3. [Free online quantum learning opportunities](https://quantumapalooza.com/)
4. [Videos on basic QIS concepts](https://www.quantumworlddetangled.com/)
5. [Statistics on QIS technology](https://qisdata.com/)

<a id="optional"></a>
# 5. Optional content

<a id="step5"></a>
## 5.1. Step 5: Measure

Bob now has to measure the qubits in a the random bases that he chose in part 2

In [49]:
def measure(bob_bases, encoded_qubits, backend):
    # Perform measurement on the qubits send by Alice
    # selected_measurements: 
    # encoded_qubits: list of QuantumCircuits received from Alice
    # backend: IBMQ backend, either simulation or hardware
    
    # Stores the results of Bob's measurements
    bob_bitstring = ''
    
    for i in range(len(encoded_qubits)):
        qc = encoded_qubits[i]
        
        if bob_bases[i] == "0":
            # 0 means we want to measure in Z basis
            qc.measure(0,0)

        elif bob_bases[i] == "1":
            # 1 means we want to measure in X basis
            qc.h(0)
            qc.measure(0,0)
        
        # Now that the measurements have been added to the circuit, let's run them.
        job = q.execute(qc, backend=backend, shots = 1) # increase shots if running on hardware
        results = job.result()
        counts = results.get_counts()
        measured_bit = max(counts, key=counts.get)

        # Append measured bit to Bob's measured bitstring
        bob_bitstring += measured_bit 
        
    return bob_bitstring

### Measure the qubit's that Bob recieved from Alice

We will use a simulated backend here but feel free to swap this out later in the extra material

In [50]:
sim_backend = q.Aer.get_backend('qasm_simulator')

bob_bitstring = measure(bob_bases, QUANTUM_CHANNEL, sim_backend)

<a id="step6"></a>
## 5.2. Step 6: Announce Basis

Alice announces which basis she used to encode each qubit via the `CLASSICAL_CHANNEL`

In [51]:
# Alice sends the list of bases used to create her encoded qubits to bob over the classical channel
CLASSICAL_CHANNEL = alice_bases

<a id="step7"></a>
## 5.3. Step 7: Find Symmetric Key

Now that Alice has announced what basis she used to encrypt her key, Bob can check against his list and see the places where they matched. The positions where they used the same basis are the places where they will also share the same key value!

In [52]:
def bob_compare_bases(alices_bases, bobs_bases):
    indices = []
    
    for i in range(len(alices_bases)):
        if alices_bases[i] == bobs_bases[i]:
            indices.append(i)
    return indices

#### Now that Bob knows which bases Alice used to encode the qubits, he can see where they happen to have agreed on chosen bases and send Alice a list of all the positions where they chose the same basis.

In [53]:
agreeing_bases = bob_compare_bases(CLASSICAL_CHANNEL, bob_bases)

In [54]:
# Send the list of agreeing_bases from Bob to Alice over the Classical channel
CLASSICAL_CHANNEL = agreeing_bases

#### Bob and Alice know all the positions where they used the same basis to encode and decode a qubit so now if they discard every bit that was encoded using a basis that didn't agree, they will have a shared key!

In [55]:
def construct_key_from_indices(bitstring, indices):
    key = ''
    for idx in indices:
        # For the indices where bases match, the bitstring bit is added to the key
        key = key + bitstring[idx] 
    return key

In [56]:
alice_key = construct_key_from_indices(alice_bitstring, CLASSICAL_CHANNEL)
bob_key = construct_key_from_indices(bob_bitstring, agreeing_bases)

# Preview the first 10 elements of each Key:
print("alice_key: ", alice_key[:10])
print("bob_key: ", bob_key[:10])
print("Alice's key is equal to Bob's key: ", alice_key == bob_key)

alice_key:  1000110010
bob_key:  1000110010
Alice's key is equal to Bob's key:  True


<a id="extra"></a>
# 6. Extra content (strictly optional!!) - Try it out with your own messages!

If you're interested in trying this out run some of the cells below.

First we will run the whole QKD protocol again and use the keys to encrypt one of our own messages!

In [57]:
# Step 1: Alice's Prepares encoding basis and choose a random btistring
alice_bitstring, alice_bases = select_encoding(KEY_LENGTH)
# Step 2: Bob selects bases for measurement
bob_bases = select_measurement(KEY_LENGTH)
# Step 3: Alice encodes the qubits
encoded_qubits = encode(alice_bitstring, alice_bases)
# Step 4: Alice sends the qubits
QUANTUM_CHANNEL = encoded_qubits
# Step 5: Bob measures the received qubits
bob_bitstring = measure(bob_bases, QUANTUM_CHANNEL, q.Aer.get_backend('qasm_simulator'))
# Step 6: Bob uses Alice's announced bases to see where they agreed with his decoding bases
CLASSICAL_CHANNEL = alice_bases
agreeing_bases = bob_compare_bases(CLASSICAL_CHANNEL, bob_bases)
# Bob announces where they agreed on encoding and decoding bases.
# Step 7: Alice and Bob construct their keys from 
CLASSICAL_CHANNEL = agreeing_bases
alice_key = construct_key_from_indices(alice_bitstring, agreeing_bases)
bob_key = construct_key_from_indices(bob_bitstring, agreeing_bases)

## Using your new keys:

In order to make it easier for you to try out your new keys, we wrote some helper functions called `encrypt_message` and `decrypt_message`, you dont need to worry about exactly what is going on inside of them - essentially they just take your text and convert it to binary and apply the key to it for encryption and decryption respectively.

Run the cell below to load the functions:

In [58]:
import binascii

def encrypt_message(unencrypted_string, key):
    # Convert ascii string to binary string
    bits = bin(int(binascii.hexlify(unencrypted_string.encode('utf-8', 'surrogatepass')), 16))[2:]
    bitstring = bits.zfill(8 * ((len(bits) + 7) // 8))
    # created the encrypted string using the key
    encrypted_string = ""
    for i in range(len(bitstring)):
        encrypted_string += str( (int(bitstring[i])^ int(key[i])) )
    return encrypted_string
    
def decrypt_message(encrypted_bits, key):
    # created the unencrypted string using the key
    unencrypted_bits = ""
    for i in range(len(encrypted_bits)):
        unencrypted_bits += str( (int(encrypted_bits[i])^ int(key[i])) )
    # Convert bitstring into
    i = int(unencrypted_bits, 2)
    hex_string = '%x' % i
    n = len(hex_string)
    bits = binascii.unhexlify(hex_string.zfill(n + (n & 1)))
    unencrypted_string = bits.decode('utf-8', 'surrogatepass')
    return unencrypted_string

How does encryption and decryption work? Once a key has been distributed between Alice and Bob, Alice uses the key to encrypt the message, and Bob uses the same key to decrypt it. To encrypt the message, Alice uses a **reversible** function, so that Bob can apply the same function to the encrypted message to decrypt it. A commonly used function is XOR - each bit of the message is XOR'd with the corresponding bit of the key to encrypt it, i.e., if the message bit is $b$, the encryted message is $b \oplus 1$ or $b \oplus 0$, depending on the corresponding bit of the key. For decryption, the encryted message is XOR'd again with the same bit of the key. This process works because
$$b \oplus 1 \oplus 1 = b$$ and $$b \oplus 0 \oplus 0 = b$$ 
Therefore, Bob recovers the original bit as long as Bob's key matches Alice's. We will use this method for encryption and decryption. This is what the functions above implement.

Now that we know how the key will be used, let's discuss how it will be distributed between Alice and Bob.

## Run the test to see how it works and then try your own strings!
To use them pass in your text e.g. `"QKD is cool!"` as the string and your key that was distributed to both Alice and Bob earlier.

In [59]:
message = "QKD is cool!"
print("Original Messge:", message)
encrypted_message = encrypt_message(message, alice_key)
print("Encrypted message:", encrypted_message)
decrypted_message = decrypt_message(encrypted_message, bob_key)
print("Decrypted message:", decrypted_message)

Original Messge: QKD is cool!
Encrypted message: 111001111100111010100110101110000010110001101110111000110101100100111001110011001100100111100111
Decrypted message: QKD is cool!


<a id="challenge"></a>
## 6.1 Challenge Problem: Implementing a function for Eve

Now that you know how the protocol works and you have seen the code for each step, can you write a function to implement an eavesdropper Eve?

As a hint, here's how Eve might eavesdrop: Eve intercepts the qubits Alice is sending, performs a measurement of them in randomly selected bases, and then sends the resulting qubits to Bob, who implements his steps of the protocol as usual. In this case, Alice and Bob will perform step 8: Analysis. How might this step be implemented? Test out this step with and without Eve, and check if Alice and Bob are able to detect Eve's presence.

Are there other ways to eavesdrop? Can you implement them in code? Will Alice and Bob be able to detect these other ways of eavesdropping?

Since this question is quite open-ended, there isn't one 'correct' way to solve it. Therefore, instead of the instruction team providing a 'solution', we encourage you to post your implementations to Piazza to discuss with other students and instructors! We're excited to read what you come up with :)